In [1]:
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [2]:
argl = [i.rstrip("\n") for i in open("proj1_parameter-file.txt", "r")]
argl

['MIS(10) = 0.43',
 'MIS(20) = 0.30',
 'MIS(30) = 0.30',
 'MIS(40) = 0.40',
 'MIS(50) = 0.40',
 'MIS(60) = 0.30',
 'MIS(70) = 0.20',
 'MIS(80) = 0.20',
 'MIS(90) = 0.20',
 'MIS(100) = 0.10',
 'MIS(120) = 0.20',
 'MIS(140) = 0.15',
 'SDC = 0.1',
 'cannot_be_together: {20, 40}, {70, 80}',
 'must-have: 20 or 40 or 50']

In [3]:
ms = dict()
for i in argl:
    if i.startswith("MIS"):
        j = i.split(" = ")
        ms.update({int(j[0][4:-1]): float(j[1])})
    elif i.startswith("SDC"):
        sdc = float(i.split("=")[1])
    elif i.startswith("cannot_be_together"):
        x_exclude = [list(j) for j in eval(i.split(":")[1])]
    elif i.startswith("must"):
        x_must = [int(j) for j in i.split(":")[1].split("or")]
ms, sdc, x_exclude, x_must

({10: 0.43,
  20: 0.3,
  30: 0.3,
  40: 0.4,
  50: 0.4,
  60: 0.3,
  70: 0.2,
  80: 0.2,
  90: 0.2,
  100: 0.1,
  120: 0.2,
  140: 0.15},
 0.1,
 [[40, 20], [80, 70]],
 [20, 40, 50])

In [4]:
ms = pd.Series(ms, name = "MIS").sort_values().reset_index()
ms

,index,MIS
0,100,0.10
1,140,0.15
2,70,0.20
3,80,0.20
4,90,0.20
5,120,0.20
6,20,0.30
7,30,0.30
8,60,0.30
9,40,0.40


In [5]:
id_dict = ms["index"].to_dict()
id_dict_inv = {val: key for key, val in id_dict.items()}
id_dict, id_dict_inv

({0: 100,
  1: 140,
  2: 70,
  3: 80,
  4: 90,
  5: 120,
  6: 20,
  7: 30,
  8: 60,
  9: 40,
  10: 50,
  11: 10},
 {10: 11,
  20: 6,
  30: 7,
  40: 9,
  50: 10,
  60: 8,
  70: 2,
  80: 3,
  90: 4,
  100: 0,
  120: 5,
  140: 1})

In [6]:
ms_dict = ms["MIS"].to_dict()
ms_dict

{0: 0.10000000000000001,
 1: 0.14999999999999999,
 2: 0.20000000000000001,
 3: 0.20000000000000001,
 4: 0.20000000000000001,
 5: 0.20000000000000001,
 6: 0.29999999999999999,
 7: 0.29999999999999999,
 8: 0.29999999999999999,
 9: 0.40000000000000002,
 10: 0.40000000000000002,
 11: 0.42999999999999999}

In [7]:
s = pd.read_csv("proj1_input-data.txt", header = None, sep = "\t",squeeze = True)
s

0                   {20, 30, 80, 70, 50, 90}
1                           {20, 10, 80, 70}
2                               {10, 20, 80}
3                               {20, 30, 80}
4                                   {20, 80}
5    {20, 30, 80, 70, 50, 90, 100, 120, 140}
Name: 0, dtype: object

In [8]:
da = s.str[1:-1].str.get_dummies(sep = ", ").rename(columns = lambda x: id_dict_inv[int(x)]).reindex(columns = ms.index, fill_value = 0).values
da

array([[0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]])

In [9]:
def mis(L):
    op = [min([ms_dict[j] for j in i]) for i in L]
    return(op)

In [10]:
def sup(L):
    op = np.vstack(da[:, i].all(axis = 1) for i in L).mean(axis = 1)
    return(op)

In [13]:
Li = (da.mean(axis = 0) > ms["MIS"].values).argmax()
L = np.array([ms.loc[Li:, "MIS"].index[da[:, Li:].mean(axis = 0) > ms["MIS"][Li]]]).T
Li, L

(0, array([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [10],
        [11]]))

In [14]:
sup(L), mis(L)

(array([ 0.16666667,  0.16666667,  0.5       ,  1.        ,  0.33333333,
         0.16666667,  1.        ,  0.5       ,  0.33333333,  0.33333333]),
 [0.10000000000000001,
  0.14999999999999999,
  0.20000000000000001,
  0.20000000000000001,
  0.20000000000000001,
  0.20000000000000001,
  0.29999999999999999,
  0.29999999999999999,
  0.40000000000000002,
  0.42999999999999999])

In [15]:
F = [L[sup(L) > mis(L)]]
F

[array([[0],
        [1],
        [2],
        [3],
        [4],
        [6],
        [7]])]

In [16]:
def pair_sup_mis(x):
    x_t = x[:, np.newaxis]
    x_sup = sup(x_t)
    x_mis = mis(x_t)
    x_sup_t = x_sup[:, np.newaxis]
    C = sp.coo_matrix(np.triu((x_sup_t >= x_mis).T & (np.abs(x_sup_t - x_sup) < sdc), 1)).nonzero()
    op = np.array([x[i] for i in C]).T
    return(op)
def append_set(C, base):
    op = np.hstack([np.tile(base, (C.shape[0], 1)), C])
    return(op)
def reduce_candidate(C):
    L_id = pd.DataFrame({0: 0}, pd.MultiIndex.from_tuples(F[-1].tolist()))
    C_id = pd.MultiIndex.from_tuples(C.tolist())
    op = C[~np.hstack([pd.DataFrame(index = C_id.droplevel(i)).join(L_id).values for i in range(C.shape[1])]).any(axis = 1)]
    return(op)
def l2_candidate_gen(L):
    C = pair_sup_mis(L.T[0])
    C = C[np.in1d(C[:,0], F[0])]
    return(C)

In [17]:
C = l2_candidate_gen(L)
C

array([[ 0,  1],
       [ 0,  5],
       [ 1,  5],
       [ 2,  7],
       [ 3,  6],
       [ 4, 10],
       [ 4, 11]])

In [18]:
F[-1]

array([[0],
       [1],
       [2],
       [3],
       [4],
       [6],
       [7]])

In [19]:
F.append(C[sup(C) >= mis(C)])
F[-1]

array([[ 0,  1],
       [ 0,  5],
       [ 1,  5],
       [ 2,  7],
       [ 3,  6],
       [ 4, 10]])

In [20]:
Ls = pd.Series(F[-1][:, -1], pd.MultiIndex.from_tuples(F[-1][:, :-1].tolist()))
Ls

0     1
0     5
1     5
2     7
3     6
4    10
dtype: int64

In [21]:
C = np.vstack([append_set(pair_sup_mis(group.values), name) for name, group in Ls.groupby(level = np.arange(F[-1].shape[1] - 1))])
C.shape

(1, 3)

In [22]:
reduce_candidate(C)

array([[0, 1, 5]])

In [23]:
[[[id_dict[k] for k in j] for j in i] for i in F]

[[[100], [140], [70], [80], [90], [20], [30]],
 [[100, 140], [100, 120], [140, 120], [70, 30], [80, 20], [90, 50]]]